In [1]:
import torch
import numpy as np
from copy import copy, deepcopy
from tqdm import tqdm

## Tug of War 2 lanes No FIFO

In [ ]:
maker_cost = {
    'Marine T' : 50,
    'Baneling T' : 75,
    'Immortal T' : 200,
    'Marine B' : 50,
    'Baneling B' : 75,
    'Immortal B' : 200,
}
def get_big_A_spend_all(miner, 
              all_A_vectors = None, vector = None, move = 0):
    if all_A_vectors is None:
        all_A_vectors = list()
    if vector is None:
        vector = (0,0,0,0,0,0)
    if miner == 0:
        all_A_vectors.append(vector)
        return list(all_A_vectors)
    
    next_vector = copy(vector)
#     if miner < 50:
#         all_A_vectors.append(vector)
#         return list(all_A_vectors)
        
#     next_vector = copy(vector)
#     get_big_A_spend_all(miner - miner, all_A_vectors, next_vector)
    if miner >= maker_cost['Marine T']:
        if move <= 0:
            next_vector = (vector[0] + 1, vector[1],vector[2],
                            vector[3], vector[4], vector[5])
            get_big_A_spend_all(miner - maker_cost['Marine T'], all_A_vectors, next_vector, 0)
        if move <= 1:
            next_vector = (vector[0], vector[1],vector[2],
                            vector[3] + 1, vector[4], vector[5])
            get_big_A_spend_all(miner - maker_cost['Marine B'], all_A_vectors, next_vector, 1)
            
        if miner >= maker_cost['Baneling T']:
            if move <= 2:
                next_vector = (vector[0], vector[1] + 1,vector[2],
                                vector[3], vector[4], vector[5])
                get_big_A_spend_all(miner - maker_cost['Baneling T'], all_A_vectors, next_vector, 2)
            if move <= 3:
                next_vector = (vector[0], vector[1],vector[2],
                                vector[3], vector[4] + 1, vector[5])
                get_big_A_spend_all(miner - maker_cost['Baneling B'], all_A_vectors, next_vector, 3)
                
            if miner >= maker_cost['Immortal T']:
                if move <= 4:
                    next_vector = (vector[0], vector[1],vector[2] + 1,
                                    vector[3], vector[4], vector[5])
                    get_big_A_spend_all(miner - maker_cost['Immortal T'], all_A_vectors, next_vector, 4)
                if move <= 5:
                    next_vector = (vector[0], vector[1],vector[2],
                                    vector[3], vector[4], vector[5] + 1)
                    get_big_A_spend_all(miner - maker_cost['Immortal B'], all_A_vectors, next_vector, 5)

    return list(all_A_vectors)

In [ ]:
# action_space = get_big_A_spend_all(225)
# print(len(action_space))
# for action in action_space:
#     print(action)

all_actions = []
mineral = 0
maker_cost_np = np.zeros(len(maker_cost))
for i, mc in enumerate(maker_cost.values()):
    maker_cost_np[i] = mc
for i in tqdm(range(1500 // 25 + 1)):
    all_actions.extend(get_big_A_spend_all(mineral))
    mineral += 25
    if mineral > 1500:
        break


In [ ]:
all_actions = np.array(all_actions)
pylon_action = np.zeros((all_actions.shape[0], 1))
print(all_actions.shape)
print(pylon_action.shape)
all_actions = np.hstack((all_actions, pylon_action))
print(all_actions.shape)
print(all_actions)
# print(all_actions[all_actions[:,5] == 7])

In [ ]:
key_mineral = 50
action_dict = {0: 1, 25: 1}
for i, a in tqdm(enumerate(np.array(all_actions[1:]))):
#     if key_mineral < 500:
#         print(np.sum(maker_cost_np * a))
    if np.sum(maker_cost_np * a[:-1]) != key_mineral:
        action_dict[key_mineral] = i + 1
        key_mineral += 25
action_dict[key_mineral] = i + 2
print(action_dict)
action_1500_dict = {}
all_actions_torch = torch.Tensor(all_actions)
action_1500_dict['actions'] = all_actions_torch
action_1500_dict['mineral'] = action_dict

torch.save(action_1500_dict, 'action_1500_dict_2L.pt')

## Tug of War 2 lanes No FIFO (One lane one time)

In [ ]:
maker_cost = {
    'Marine' : 50,
    'Baneling' : 75,
    'Immortal' : 200,
#     'Plyon' : 300
}
def get_big_A_spend_all_olot(miner, 
              all_A_vectors = None, vector = None, move = 0, pylons = 0):
    if all_A_vectors is None:
        all_A_vectors = list()
    if vector is None:
        vector = (0,0,0,0)
    if miner == 0:
        all_A_vectors.append(vector) 
        return list(all_A_vectors)
    pylon_cost = 300 + pylons * 100
    next_vector = copy(vector)
#     if miner < 50:
#         all_A_vectors.append(vector)
#         return list(all_A_vectors)
        
#     next_vector = copy(vector)
#     get_big_A_spend_all_olot(miner - miner, all_A_vectors, next_vector)
    if miner >= maker_cost['Marine']:
        if move <= 0:
            next_vector = (vector[0] + 1, vector[1],vector[2], vector[3])
            get_big_A_spend_all_olot(miner - maker_cost['Marine'], all_A_vectors, next_vector, 0,
                                     pylons = pylons)
            
        if miner >= maker_cost['Baneling']:
            if move <= 1:
                next_vector = (vector[0], vector[1] + 1,vector[2], vector[3])
                get_big_A_spend_all_olot(miner - maker_cost['Baneling'], all_A_vectors, next_vector, 1,
                                         pylons = pylons)
                
            if miner >= maker_cost['Immortal']:
                if move <= 2:
                    next_vector = (vector[0], vector[1],vector[2] + 1, vector[3])
                    get_big_A_spend_all_olot(miner - maker_cost['Immortal'], all_A_vectors, next_vector, 2,
                                             pylons = pylons)
                if miner >= pylon_cost and pylons < 3:
                    if move <= 3:
                        next_vector = (vector[0], vector[1], vector[2], vector[3] + 1)
                        get_big_A_spend_all_olot(miner - pylon_cost, all_A_vectors, next_vector, 3, 
                                                 pylons = pylons + 1)

    return list(all_A_vectors)

In [ ]:
# action_space = get_big_A_spend_all_olot(1500)
# print(len(action_space))
# for action in action_space:
#     print(action)
maker_cost_np = np.zeros(len(maker_cost))
for i, mc in enumerate(maker_cost.values()):
    maker_cost_np[i] = mc
    
all_actions = {}
for i in range(4):
    actions = []
    mineral = 0
    for _ in tqdm(range(1500 // 25 + 1)):
        actions.extend(get_big_A_spend_all_olot(mineral, pylons = i))
        mineral += 25
        if mineral > 1500:
            break
    
    print(len(actions))
    all_actions[i] = np.array(actions)

In [ ]:
for i in range(4):
    print(i)
    print(all_actions[i][-20:])

In [ ]:
pylon_cost = {
    0:{
        0: 0,
        1: 300,
        2: 700,
        3: 1200
    },
    1:{
        0: 0,
        1: 400,
        2: 900
    },
    2:{
        0: 0,
        1: 500
    },
    3: {
        0: 0,
    }
}
all_action_dict = {}
for j in range(4):
    action_pylon = all_actions[j]
    all_action_dict[j] = {0: 1, 25: 1}
    key_mineral = 50
    for i, a in tqdm(enumerate(np.array(action_pylon[1:]))):
        pylon_c = pylon_cost[j][a[-1]]
#         print(pylon_c)
#         print(a)
#         if key_mineral < 500:
#         print(np.sum(maker_cost_np * a[:-1]) + pylon_c)
        if np.sum(maker_cost_np * a[:-1]) + pylon_c != key_mineral:
            all_action_dict[j][key_mineral] = i + 1
            key_mineral += 25
            
    all_action_dict[j][key_mineral] = i + 2
    print(all_action_dict)
    
action_1500_dict = {}
action_1500_dict['actions'] = all_actions
action_1500_dict['mineral'] = all_action_dict

torch.save(action_1500_dict, 'action_1500_dict_2L.pt')

In [ ]:
print(action_1500_dict)

In [ ]:
a = np.array([1,2,3])


## Generate Marked experience.

In [3]:
rand_exp = torch.load("rand_v_rand.pt")
agents_exp = torch.load("all_experiences.pt")
unmark_exp = rand_exp[:96600] + agents_exp
print(len(rand_exp))
print(len(agents_exp))
print(len(unmark_exp))
torch.save(unmark_exp, "all_experiences_100000.pt")

137200
3400
100000


In [4]:
# unmark_exp = torch.load("test_random_vs_random_2l.pt")
def player_1_win_condition(state_1_T_hp, state_1_B_hp, state_2_T_hp, state_2_B_hp):
    if min(state_1_T_hp, state_1_B_hp) == min(state_2_T_hp, state_2_B_hp):
        if state_1_T_hp + state_1_B_hp == state_2_T_hp + state_2_B_hp:
            return 0
        elif state_1_T_hp + state_1_B_hp > state_2_T_hp + state_2_B_hp:
            return 1
        else:
            return -1
    else:
        if min(state_1_T_hp, state_1_B_hp) > min(state_2_T_hp, state_2_B_hp):
            return 1
        else:
            return -1
mark_exp = deepcopy(unmark_exp)

In [5]:
insert_num = 0
for i, exp in enumerate(unmark_exp):
#     print(exp)
    if i != 0 and exp[0][-1] == 2:
        s1_hp_t, s1_hp_b, s2_hp_t, s2_hp_b = unmark_exp[i - 1][1][27:31]
        
        win_lose = player_1_win_condition(s1_hp_t, s1_hp_b, s2_hp_t, s2_hp_b)
        mark_exp.insert(i + insert_num, win_lose)
#         skip = True
        insert_num += 1
#         print(s1_hp_t, s1_hp_b, s2_hp_t, s2_hp_b)
#         print(win_lose)


In [6]:
# for i, exp in enumerate(mark_exp):
#     print(exp)
#     if type(exp) == type(int(1)):
#         print(list(mark_exp[i - 1][1][27:31]))

In [7]:
torch.save(mark_exp, "all_experiences_100000_mark.pt")

In [ ]:
print(mark_exp)